In [ ]:
# Demo: GPT-Powered CV Evaluation Based on Job Description

import os
import fitz  # PyMuPDF
import json
import pandas as pd
from openai import OpenAI

# === Replace with your actual API key during local testing ===
client = OpenAI(api_key="your-api-key-here")

# --- Step 1: Extract Text from PDFs ---
def extract_text_from_folder(folder_path):
    extracted = []
    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            path = os.path.join(folder_path, file)
            doc = fitz.open(path)
            text = "\n".join(page.get_text() for page in doc)
            extracted.append({"filename": file, "content": text})
    return extracted

# --- Step 2: Normalize Job Description ---
def normalize_job_description(jd_text):
    structured_format = {
        "JobTitle": "", "Department": "", "ReportsTo": "", "JobPurpose": "",
        "KeyResponsibilities": [], "WorkModality": "", "AdditionalDetails": "",
        "Qualifications": {
            "education": [], "work_experience": [], "skills": [],
            "certifications_and_courses": [], "languages": [{"language": "", "proficiency": ""}],
            "systems_knowledge": []
        }
    }
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": f"Summarize this job description into structured format: {jd_text}"},
            {"role": "user", "content": json.dumps(structured_format)}
        ]
    )
    return json.loads(response.choices[0].message.content)

# --- Step 3: Evaluate CVs Based on JD ---
def evaluate_cv(cv_text, job_description):
    evaluation_template = {
        "dimensions": [
            {"name": "education", "value": 0, "reasoning": ""},
            {"name": "work_experience", "value": 0, "reasoning": ""},
            {"name": "skills", "value": 0, "reasoning": ""},
            {"name": "certifications_and_courses", "value": 0, "reasoning": ""},
            {"name": "languages", "value": 0, "reasoning": ""},
            {"name": "systems_knowledge", "value": 0, "reasoning": ""},
            {"name": "general_assessment", "value": 0, "reasoning": ""}
        ]
    }
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": f"Job Description:\n{json.dumps(job_description)}"},
            {"role": "system", "content": f"Candidate CV:\n{cv_text}"},
            {"role": "user", "content": f"Evaluate based on this structure:\n{json.dumps(evaluation_template)}"}
        ]
    )
    return json.loads(response.choices[0].message.content)

# --- Step 4: Run Evaluation Process ---
def run_pipeline(cv_folder, jd_text, output_excel_path="evaluation_results.xlsx"):
    cvs = extract_text_from_folder(cv_folder)
    jd = normalize_job_description(jd_text)
    all_results = []

    for cv in cvs:
        result = {"filename": cv["filename"]}
        eval_data = evaluate_cv(cv["content"], jd)
        for dim in eval_data["dimensions"]:
            result[f"{dim['name']}_score"] = dim["value"]
            result[f"{dim['name']}_reasoning"] = dim["reasoning"]
        all_results.append(result)
        print(f"✔ Processed {cv['filename']}")

    df = pd.DataFrame(all_results)
    df.to_excel(output_excel_path, index=False)
    print("🎯 All evaluations saved to:", output_excel_path)

# --- Example Usage ---
# run_pipeline("path/to/cv_folder", "Paste your job description text here")
